### 1. From HTML

*Using only beautiful soap, no regex*

Save in a dataframe the next information using web scraping. Each row of the dataframe must have in different columns:

- The name of the title
- The id of the div where is the value scraped. If there is not id, then the value is must be numpy.nan
- The name of the tag where is the value scraped.
- The next scraped values in different rows: 
    - The value: "Este es el segundo párrafo"  --> Row 1
    - The url https://pagina1.xyz/ --> Row 2
    - The url https://pagina4.xyz/ --> Row 3
    - The url https://pagina5.xyz/ --> Row 4
    - The value "links footer-links" --> Row 5
    - The value "Este párrafo está en el footer" --> Row 6

In [4]:
html = """<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Página de prueba</title>
</head>
<body>
<div id="main" class="full-width">
    <h1>El título de la página</h1>
    <p>Este es el primer párrafo</p>
    <p>Este es el segundo párrafo</p>
    <div id="innerDiv">
        <div class="links">
            <a href="https://pagina1.xyz/">Enlace 1</a>
            <a href="https://pagina2.xyz/">Enlace 2</a>
        </div>
        <div class="right">
            <div class="links">
                <a href="https://pagina3.xyz/">Enlace 3</a>
                <a href="https://pagina4.xyz/">Enlace 4</a>
            </div>
        </div>
    </div>
    <div id="footer">
        <!-- El footer -->
        <p>Este párrafo está en el footer</p>
        <div class="links footer-links">
            <a href="https://pagina5.xyz/">Enlace 5</a>
        </div>
    </div>
</div>
</body>
</html>"""

In [2]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(html, 'html.parser')
soup

<html lang="es">
<head>
<meta charset="utf-8"/>
<title>Página de prueba</title>
</head>
<body>
<div class="full-width" id="main">
<h1>El título de la página</h1>
<p>Este es el primer párrafo</p>
<p>Este es el segundo párrafo</p>
<div id="innerDiv">
<div class="links">
<a href="https://pagina1.xyz/">Enlace 1</a>
<a href="https://pagina2.xyz/">Enlace 2</a>
</div>
<div class="right">
<div class="links">
<a href="https://pagina3.xyz/">Enlace 3</a>
<a href="https://pagina4.xyz/">Enlace 4</a>
</div>
</div>
</div>
<div id="footer">
<!-- El footer -->
<p>Este párrafo está en el footer</p>
<div class="links footer-links">
<a href="https://pagina5.xyz/">Enlace 5</a>
</div>
</div>
</div>
</body>
</html>

In [11]:
#Hay que obtener todos los elementos por separado para después trabajar como se desea en un DataFrame

# 1º Los links:

lista_enlaces = soup.find_all('a')

links = [link.get('href') for link in lista_enlaces]
links

link_1 = links[0]
link_4 = links[3]
link_5 = links[-1]

# 2º El título

tit_pag = soup.h1.string

titulo = [tit_pag for i in range(6)]

# 3º Los ids

web_id = [etiqueta.get('id') for etiqueta in soup.find_all('div')]
_id_final = [web_id[0], web_id[1], web_id[1], web_id[1], web_id[-2], web_id[-2]]


# 4º Los párrafos y el párrafo del footer
parrafos = [parrafo.string for parrafo in soup.find_all('p')]
parrafo_footer = parrafos[-1]

In [14]:
soup.find_all(id='footer')

[<div id="footer">
 <!-- El footer -->
 <p>Este párrafo está en el footer</p>
 <div class="links footer-links">
 <a href="https://pagina5.xyz/">Enlace 5</a>
 </div>
 </div>]

In [15]:
web_id

['main', 'innerDiv', None, None, None, 'footer', None]

In [94]:
#Ahora creo que diccionario al que iré añadiendo las keys
diccionario ={}

diccionario['Title'] = titulo
diccionario['div id'] = _id_final
diccionario['tag name'] = ['p', 'a', 'a', 'a', 'div', 'p'] 
diccionario['Scraped Value'] = [seugndo_parrafo, link_1, link_4, link_5, footer_links, parrafo_footer]

In [101]:
_id_final

['main', 'innerDiv', 'innerDiv', 'innerDiv', 'footer', 'footer']

In [96]:
df_diccionario = pd.DataFrame(diccionario)

In [102]:
df_diccionario = pd.DataFrame(diccionario)
df_diccionario.set_index('Title')

,div id,tag name,Scraped Value
Title,,,
El título de la página,main,p,Este es el segundo párrafo
El título de la página,innerDiv,a,https://pagina1.xyz/
El título de la página,innerDiv,a,https://pagina4.xyz/
El título de la página,innerDiv,a,https://pagina5.xyz/
El título de la página,footer,div,"[links, footer-links]"
El título de la página,footer,p,Este párrafo está en el footer


In [103]:
df_diccionario #Sin el set index

,Title,div id,tag name,Scraped Value
0,El título de la página,main,p,Este es el segundo párrafo
1,El título de la página,innerDiv,a,https://pagina1.xyz/
2,El título de la página,innerDiv,a,https://pagina4.xyz/
3,El título de la página,innerDiv,a,https://pagina5.xyz/
4,El título de la página,footer,div,"[links, footer-links]"
5,El título de la página,footer,p,Este párrafo está en el footer


### 2. From Amazon

*Using  beautiful soap  (regex optional)*

Save in a dataframe the next information using web scraping. Using product pages from Amazon, do the following: 

- Get the product name from the web and save it in a column called "item_name"
- Get the price from the web and save it in a column called "item_price"

While you are doing the exercise, document the steps you are doing. Try to do the program for generic pages. If you cannot do it generic, explain the reasons. 

*We recommend to get the source-code, save it in a local file and work from there. It is possible that Amazon detects that you are using webscraping and it changes the source code to avoid possibles attacks.*

-------------------------------

**Example:** 

url = https://www.amazon.es/Tommy-Hilfiger-UM0UM00054-Camiseta-Hombre/dp/B01MYD0T1F/ref=sr_1_1?dchild=1&pf_rd_p=58224bec-cac9-4dd2-a42a-61b1db609c2d&pf_rd_r=VZQ1JTQXFVRZ9E9VSKX4&qid=1595364419&s=apparel&sr=1-1

*item_name* --> "Tommy Hilfiger Logo Camiseta de Cuello Redondo,Perfecta para El Tiempo Libre para Hombre"

*item_price* --> [[18,99 € - 46,59 €]] or one of the options.


